## **ANN Group A Assignment_7: Implement Artificial Neural Network training process in Python by using Forward Propagation, Back Propagation.**

In [95]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [96]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.Wij = np.random.uniform(-0.5, 0.5, (self.input_size, self.hidden_size)) # Weights from input to hidden layer
        self.Vjk = np.random.uniform(-0.5, 0.5, (self.hidden_size, self.output_size)) # Weights from hidden to output layer
        
        self.bias_j = np.random.uniform(-0.5, 0.5, (1, self.hidden_size)) # hidden layer bias
        self.bias_k = np.random.uniform(-0.5, 0.5, (1, self.output_size)) # output  layer bias
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
     
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_prop(self, x):
        self.hidden_layer_input = np.dot(x, self.Wij) + self.bias_j # weighted sum  of inputs to the hidden layer
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input) # output  of the hidden layer
        
        self.output_layer_input = np.dot(self.hidden_layer_output, self.Vjk) + self.bias_k
        self.output_layer_output = self.sigmoid(self.output_layer_input)
        
        return self.output_layer_output
    
    def backprop(self, x, y, output):
        error = y - output
        delta_output = error * self.sigmoid_derivative(output)
        
        hidden_layer_error = delta_output.dot(self.Vjk.T)
        hidden_layer_delta = hidden_layer_error * self.sigmoid_derivative(self.hidden_layer_output)
        
        # weight adjustment between hidden and output layers
        self.Vjk += self.hidden_layer_output.T.dot(delta_output) * self.learning_rate
         
       # calculate new bias value for the output layer
        self.bias_k += np.sum(delta_output) * self.learning_rate
         
        # weight adjustment between input and hidden layers
        self.Wij += x.T.dot(hidden_layer_delta) * self.learning_rate
        
        # calculate new bias value for the hidden layer
        self.bias_j += np.sum(hidden_layer_delta) * self.learning_rate
        
    def train(self, x, y, epochs):
        for epoch in range(epochs):
            output = self.forward_prop(x)
            self.backprop(x, y, output)
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}: Error {np.mean(np.abs(y - output))}')

In [97]:
df = pd.read_csv('Admission_Predict_Ver1.1.csv')
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [98]:
df.shape

(500, 9)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          500 non-null    int64  
 2   TOEFL Score        500 non-null    int64  
 3   University Rating  500 non-null    int64  
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 35.3 KB


In [100]:
df.duplicated().sum()

0

In [101]:
df.drop(columns=['Serial No.'],inplace=True)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [102]:
# Binarizing 'Chance of Admission' column
df['Chance of Admit '] = df['Chance of Admit '].apply(lambda x: 1 if x >= 0.5 else 0)
df.head()


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,1
1,324,107,4,4.0,4.5,8.87,1,1
2,316,104,3,3.0,3.5,8.00,1,1
3,322,110,3,3.5,2.5,8.67,1,1
4,314,103,2,2.0,3.0,8.21,0,1


In [103]:
X = df.drop(columns=['Chance of Admit '])
y =  df[['Chance of Admit ']]


In [104]:

X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1
496,337,117,5,5.0,5.0,9.87,1
497,330,120,5,4.5,5.0,9.56,1
498,312,103,4,4.0,5.0,8.43,0


In [105]:
y

,Chance of Admit
0,1
1,1
2,1
3,1
4,1
...,...
495,1
496,1
497,1
498,1


In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [107]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [108]:
X_train_scaled

array([[-0.53736015, -0.51949116, -0.05463584, ...,  0.00933125,
        -0.32658176, -1.11114215],
       [ 0.16363964,  0.44925692, -1.8029826 , ...,  0.00933125,
        -0.04593523,  0.89997486],
       [-1.41360989, -1.0038652 , -0.05463584, ..., -1.05709751,
        -1.13550409, -1.11114215],
       ...,
       [ 0.77701445, -0.03511712, -0.05463584, ...,  0.00933125,
         0.89505605,  0.89997486],
       [ 0.86463943,  0.61071493,  1.69371093, ...,  0.54254563,
         1.09315948,  0.89997486],
       [-1.41360989, -0.35803314, -1.8029826 , ..., -1.59031189,
        -1.26757304, -1.11114215]])

In [109]:
X_test_scaled

array([[-0.27448523, -0.19657513, -0.92880922, -0.84504549, -1.59031189,
        -0.22753004, -1.11114215],
       [-0.3621102 , -0.03511712, -0.05463584, -0.3523076 , -0.52388313,
        -0.17800418,  0.89997486],
       [ 2.00376408,  1.90237904,  1.69371093,  1.1259061 ,  0.54254563,
         1.86906457, -1.11114215],
       [-0.01161031, -0.03511712, -0.92880922,  0.1404303 ,  0.00933125,
         0.11915096,  0.89997486],
       [ 0.77701445,  0.44925692,  0.81953755,  1.1259061 ,  0.54254563,
         0.64742677,  0.89997486],
       [-0.44973518, -1.0038652 , -0.92880922, -0.84504549,  0.00933125,
        -0.37610761,  0.89997486],
       [ 0.33888959, -0.51949116, -0.05463584, -0.3523076 ,  0.00933125,
         0.28423715,  0.89997486],
       [ 0.9522644 , -0.68094917, -0.05463584,  0.6331682 ,  0.54254563,
        -0.44214209,  0.89997486],
       [ 0.42651456,  0.61071493, -0.05463584,  0.1404303 ,  0.54254563,
         0.43281472,  0.89997486],
       [ 0.16363964, -0.1965

In [112]:
encoder = OneHotEncoder(categories='auto')
y_train_onehot = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()

# Defining neural network parameters
input_size = X_train_scaled.shape[1]
hidden_size = 1000  # Change this value as needed
output_size = y_train_onehot.shape[1]
learning_rate = 0.1
epochs = 10000

# Training the neural network
obj_ANN = ANN(input_size, hidden_size, output_size, learning_rate)
obj_ANN.train(X_train_scaled, y_train_onehot, epochs)

# Making predictions on the test set
for i in range(len(X_test_scaled)):
    predicted = obj_ANN.forward_prop(X_test_scaled[i])
    predicted_class = np.argmax(predicted)
    actual_class = y_test.values[i]
    print(f"Predicted: {predicted_class}\tActual: {actual_class}")

Epoch 0: Error 0.492509302617874
Epoch 1000: Error 0.07160515757396518
Epoch 2000: Error 0.0775
Epoch 3000: Error 0.0775
Epoch 4000: Error 0.0775
Epoch 5000: Error 0.0775
Epoch 6000: Error 0.0775
Epoch 7000: Error 0.0775
Epoch 8000: Error 0.0775
Epoch 9000: Error 0.0775
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [0]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Predicted: 1	Actual: [1]
Pred

In [119]:
# Scaling the test data
X_test_scaled = scaler.transform(X_test)

# Calculate accuracy
correct_count = 0
total = len(X_test_scaled)  # Use the scaled test set

for i in range(total):
    predicted = obj_ANN.forward_prop(X_test_scaled[i])
    predicted_class = np.argmax(predicted)
    if predicted_class == y_test.values[i]:  # Compare with binarized values
        correct_count += 1

accuracy = (correct_count / total) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 94.00%
